In [2]:
import numpy as np
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt

In [4]:
img_path = "test1.jpg"

In [3]:
model = YOLO("best.pt")

In [5]:
detections = model(img_path)[0]


image 1/1 e:\My Projects\Number Plate detection\test1.jpg: 640x480 1 Back Plate, 77.0ms
Speed: 4.0ms preprocess, 77.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 480)


In [6]:
detections_ = []
for detection in detections.boxes.data.tolist():
    x1, y1, x2, y2, score, class_id = detection
    detections_.append([x1, y1, x2, y2, score])

In [ ]:
fls = detections_[0]
fls